In [50]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression

In [51]:
x_train = pd.read_csv('Downloads\cmf-2022-bank-scoring-case\X_train.csv')
test = pd.read_csv('Downloads\cmf-2022-bank-scoring-case\X_test.csv')
y_sample = pd.read_csv('Downloads\cmf-2022-bank-scoring-case\y_test_sample.csv')
y_train = pd.read_csv('Downloads\cmf-2022-bank-scoring-case\y_train.csv')

In [52]:
test

,index,credit_card_utilization,age,overdue_30_59_days,debt_to_income,monthly_income,credits_loans,overdue__more_than_90_days,mortgage_loans,overdue_60_89_days,family_members
0,143522,0.021069,60,0,0.184907,14403.715440,9,0,1,0,0.0
1,116747,0.009985,82,0,0.446264,6768.581963,6,0,3,0,0.0
2,66075,0.535187,43,1,0.406317,4686.080429,14,0,2,0,3.0
3,133292,0.051264,31,0,0.311893,3867.175176,8,0,0,0,1.0
4,83134,0.069525,45,0,0.294956,9869.273012,6,0,1,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
38871,44475,0.731740,39,0,0.251005,3474.894386,5,0,0,0,0.0
38872,103189,0.010959,70,0,0.152539,4170.426464,14,0,1,0,0.0
38873,48732,0.084612,52,0,5288.053772,NaN,15,0,2,0,0.0
38874,135410,0.741717,51,0,0.378356,5441.458787,9,0,2,0,0.0


In [53]:
indexes = test['index']

In [54]:
test.drop(['index'], axis = 1, inplace = True)


In [55]:
x_train['target'] = y_train
x_train.drop(x_train[x_train.age < 18].index, inplace = True)
x_train.drop(x_train[x_train.monthly_income > 1000000].index, inplace = True)


In [56]:
y = x_train.target
x_train.drop(['target'], axis = 1, inplace = True)

In [57]:
x_train['Test'] = False
test['Test'] = True

In [58]:
data = pd.concat([x_train, test], axis = 0)
data

,credit_card_utilization,age,overdue_30_59_days,debt_to_income,monthly_income,credits_loans,overdue__more_than_90_days,mortgage_loans,overdue_60_89_days,family_members,Test
0,0.000000,56,0,0.504328,6823.894458,9,0,1,0,5.0,False
1,0.170659,45,1,2.405860,356.580765,5,0,0,0,0.0,False
2,0.342176,75,1,0.445066,20349.895825,9,0,3,0,0.0,False
3,0.046219,50,0,0.814508,2894.368968,12,0,1,0,3.0,False
4,0.679556,58,0,0.390311,24833.227614,15,0,4,0,2.0,False
...,...,...,...,...,...,...,...,...,...,...,...
38871,0.731740,39,0,0.251005,3474.894386,5,0,0,0,0.0,True
38872,0.010959,70,0,0.152539,4170.426464,14,0,1,0,0.0,True
38873,0.084612,52,0,5288.053772,NaN,15,0,2,0,0.0,True
38874,0.741717,51,0,0.378356,5441.458787,9,0,2,0,0.0,True


In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143982 entries, 0 to 38875
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   credit_card_utilization     143982 non-null  float64
 1   age                         143982 non-null  int64  
 2   overdue_30_59_days          143982 non-null  int64  
 3   debt_to_income              143982 non-null  float64
 4   monthly_income              115444 non-null  float64
 5   credits_loans               143982 non-null  int64  
 6   overdue__more_than_90_days  143982 non-null  int64  
 7   mortgage_loans              143982 non-null  int64  
 8   overdue_60_89_days          143982 non-null  int64  
 9   family_members              140222 non-null  float64
 10  Test                        143982 non-null  bool   
dtypes: bool(1), float64(4), int64(6)
memory usage: 12.2 MB


In [60]:
data.monthly_income = data.monthly_income.fillna(data.monthly_income.mean())
data.family_members = data.family_members.fillna(data.family_members.mean())

In [61]:
train = data[data.Test == 0].reset_index(drop = True)
test = data[data.Test == 1].reset_index(drop = True)

In [62]:
train.drop(['Test'], axis = 1, inplace = True)
test.drop(['Test'], axis = 1, inplace = True)

In [63]:
x = train.copy()
#y = y_train

In [64]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [65]:
scaler.fit(x)

StandardScaler()

In [66]:
scaled_features = scaler.transform(x)

In [67]:
scaled_data = pd.DataFrame(scaled_features, columns = x.columns)


In [68]:
x = scaled_data

In [69]:
test = scaler.transform(test)

In [70]:
x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x, y, test_size = 0.25)

In [71]:
model = KNeighborsClassifier(n_neighbors = 4)

In [72]:
model.fit(x_training_data, y_training_data)

KNeighborsClassifier(n_neighbors=4)

In [73]:
predictions = model.predict(x_test_data)

In [74]:

print(classification_report(y_test_data, predictions))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97     24552
           1       0.53      0.07      0.13      1725

    accuracy                           0.93     26277
   macro avg       0.73      0.53      0.55     26277
weighted avg       0.91      0.93      0.91     26277



In [75]:
predictions = model.predict_proba(test)


In [76]:
answers = pd.DataFrame()
answers['index'] = indexes
answers['target'] = predictions[:,1]
answers

,index,target
0,143522,0.00
1,116747,0.00
2,66075,0.00
3,133292,0.25
4,83134,0.00
...,...,...
38871,44475,0.00
38872,103189,0.00
38873,48732,0.00
38874,135410,0.00


In [77]:
#linear

In [78]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [79]:
mod = LinearRegression()

In [80]:
mod.fit(X_train, y_train)

LinearRegression()

In [81]:
y_pre=model.predict(X_test)
y_pre

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [82]:
mean_absolute_error(y_test, y_pre)

NameError: name 'mean_absolute_error' is not defined

In [ ]:
y_pre=model.predict_proba(test)

In [ ]:
answers_l = pd.DataFrame()
answers_l['index'] = indexes
answers_l['target'] = y_pre[:,1]
answers_l

In [ ]:
#random forest

In [ ]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)
y_train = np.array(y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
rfc = RandomForestClassifier()
rfc.fit(X_train_scaled, y_train)
display(rfc.score(X_train_scaled, y_train))

In [ ]:
preds = rfc.predict(test)

In [ ]:
answers = pd.DataFrame()
answers['index'] = indexes
answers['target'] = preds
answers

In [ ]:
answers.to_csv('Downloads/cmf-2022-bank-scoring-case/ran.csv', index = False)